# 04 - Evaluation and Forecast
Generate rolling forecasts, compare them against actuals, and log error metrics for reporting.


## Steps
1. Load the processed sales series.
2. Produce a fresh forecast with the Prophet pipeline.
3. Align predicted and actual values for shared dates.
4. Compute MAE, RMSE, and MAPE and store them under `result/`.


In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))


In [2]:
from pathlib import Path

import pandas as pd

from src.utils import load_dataframe, save_dataframe
from src.prophet_pipeline import run_pipeline
from src.evaluate_model import evaluate_forecast_frame


In [8]:
data_path = Path("../data/processed/sales_daily.csv")

daily_df = load_dataframe(data_path)
daily_df['ds'] = pd.to_datetime(daily_df['ds'])
model_input = daily_df.rename(columns={"ds": "InvoiceDate", "y": "Sales"})
model, forecast = run_pipeline(model_input, periods=90, freq="D")

aligned = (
    forecast[["ds", "yhat", "yhat_lower", "yhat_upper"]]
    .merge(daily_df, on="ds", how="left")
    .rename(columns={"y": "y_actual"})
)
evaluation_df = aligned.rename(columns={"y_actual": "y"})

metrics = evaluate_forecast_frame(evaluation_df[["y", "yhat"]])
metrics_df = pd.DataFrame([metrics])
metrics_path = save_dataframe(metrics_df, Path("../result/performance_metrics.csv"))

print(f"Metrics saved to: {metrics_path}")
metrics_df


18:56:28 - cmdstanpy - INFO - Chain [1] start processing
18:56:28 - cmdstanpy - INFO - Chain [1] done processing


Metrics saved to: ..\result\performance_metrics.csv


,MAE,RMSE,MAPE
0,9535.62833,14311.18057,31.057873
